In [1]:
a = "hello" + " " + "word" + "!"
b = 1 + 2 + 3 + 4 + 5


print(id(1), type(1))
print(id(id), type(id))
print(id(type), type(type))


class A:
    def public(self):
        return 42

    def _private(self):
        return "test"

    def __protect(self):
        return True

    def wrapper_protect(self):
        return self.__protect()


a = A()

print(a.public())
print(a._private())
print(a._A__protect())


4558167416 <class 'int'>
4561080688 <class 'builtin_function_or_method'>
4557255408 <class 'type'>
42
test
True


In [4]:
class Singleton(object):
    def __new__(self):
        if not hasattr(self, 'instance'):
            self.instance = super(Singleton, self).__new__(self)
        return self.instance

s = Singleton()
print("Object created", s, id(s))
s1 = Singleton()
print("Object created", s1, id(s1))


def f():
	return 2 + 2


q = f

print(q())

def repair_deco(func): # тут постоянный аргумент это func
    def wrapper(a, b): # тут аргументы которые получает функция
        return func(a, b) - 1
    return wrapper

@repair_deco
def wrong_func(a, b):
	return a + b + 1

print(f"2 + 2 = {wrong_func(2, 2)}")
print(f"5 + 5 = {wrong_func(5, 5)}")

Object created <__main__.Singleton object at 0x112ce6590> 4610483600
Object created <__main__.Singleton object at 0x112ce6590> 4610483600
4
2 + 2 = 4
5 + 5 = 10


In [5]:
from datetime import datetime

def timeit(func):
	def wrapper(val):
		start = datetime.now()
		res = func(val)
		end = datetime.now()
		print(f"time: {end - start}")
		return res

	return wrapper

@timeit
def get_list_1(val):
	return [x for x in range(val) if x % 2]

@timeit
def get_list_2(val):
	new_list = []
	for x in range(val):
		if x % 2:
			new_list.append(x)
	return new_list


val = 10000000

a = get_list_1(val)
b = get_list_2(val)


time: 0:00:00.457257
time: 0:00:00.498066
